In [1]:
import pandas as pd
import numpy as np
from pyproj import Proj, transform
from shapely.geometry import Point 
import os
import geopandas as gpd
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship
import json
from geojson import Feature, FeatureCollection, Point

In [2]:
os.environ['PROJ_LIB']= "C:\\Users\\dbik\\Anaconda3\\envs\\violationproj\\Library\\share"

In [3]:
# Read the CSV file into Pandas
moving_df = pd.read_csv('Moving_Violations_Issued_in_September_2018.csv')
moving_df.head(3)

,OBJECTID,ROW_,LOCATION,ADDRESS_ID,STREETSEGID,XCOORD,YCOORD,TICKETTYPE,FINEAMT,TOTALPAID,PENALTY1,PENALTY2,ACCIDENTINDICATOR,AGENCYID,TICKETISSUEDATE,VIOLATIONCODE,VIOLATIONDESC,ROW_ID
0,14828908,NaN,600 BLK KENILWORTH AVE NE S/B,813891.0,11963.0,404478.832866,136788.897372,Moving,200,200,0,NaN,No,25.0,2018-09-01T14:04:00.000Z,T121,SPEED 21-25 MPH OVER THE SPEED LIMIT,NaN
1,14828909,NaN,5800 BLK NEW HAMPSHIRE AVE NE SW/B,800995.0,838.0,399529.150000,143757.600000,Moving,100,100,0,NaN,No,25.0,2018-09-04T12:11:00.000Z,T119,SPEED 11-15 MPH OVER THE SPEED LIMIT,NaN
2,14828910,NaN,3RD ST TUNNEL NW N/B BY MA AVE,814983.0,6364.0,398788.156100,136798.660300,Moving,150,150,0,NaN,No,25.0,2018-09-09T13:55:00.000Z,T120,SPEED 16-20 MPH OVER THE SPEED LIMIT,NaN


In [4]:
# reduce the number of columns to keep only those relevant for the analysis
mv_df = moving_df[['STREETSEGID', 'LOCATION', 'XCOORD', 'YCOORD', 'TOTALPAID', 'VIOLATIONDESC']]
mv_df.head(3)

,STREETSEGID,LOCATION,XCOORD,YCOORD,TOTALPAID,VIOLATIONDESC
0,11963.0,600 BLK KENILWORTH AVE NE S/B,404478.832866,136788.897372,200,SPEED 21-25 MPH OVER THE SPEED LIMIT
1,838.0,5800 BLK NEW HAMPSHIRE AVE NE SW/B,399529.150000,143757.600000,100,SPEED 11-15 MPH OVER THE SPEED LIMIT
2,6364.0,3RD ST TUNNEL NW N/B BY MA AVE,398788.156100,136798.660300,150,SPEED 16-20 MPH OVER THE SPEED LIMIT


In [5]:
mv_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116370 entries, 0 to 116369
Data columns (total 6 columns):
STREETSEGID      115904 non-null float64
LOCATION         116370 non-null object
XCOORD           116370 non-null float64
YCOORD           116370 non-null float64
TOTALPAID        116370 non-null int64
VIOLATIONDESC    116369 non-null object
dtypes: float64(3), int64(1), object(2)
memory usage: 5.3+ MB


In [6]:
# dropna
mv_dropna = mv_df.dropna(how='any')
mv_dropna.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115903 entries, 0 to 116369
Data columns (total 6 columns):
STREETSEGID      115903 non-null float64
LOCATION         115903 non-null object
XCOORD           115903 non-null float64
YCOORD           115903 non-null float64
TOTALPAID        115903 non-null int64
VIOLATIONDESC    115903 non-null object
dtypes: float64(3), int64(1), object(2)
memory usage: 6.2+ MB


In [7]:
mv_dropna.head(3)

,STREETSEGID,LOCATION,XCOORD,YCOORD,TOTALPAID,VIOLATIONDESC
0,11963.0,600 BLK KENILWORTH AVE NE S/B,404478.832866,136788.897372,200,SPEED 21-25 MPH OVER THE SPEED LIMIT
1,838.0,5800 BLK NEW HAMPSHIRE AVE NE SW/B,399529.150000,143757.600000,100,SPEED 11-15 MPH OVER THE SPEED LIMIT
2,6364.0,3RD ST TUNNEL NW N/B BY MA AVE,398788.156100,136798.660300,150,SPEED 16-20 MPH OVER THE SPEED LIMIT


In [8]:
location_df = mv_dropna[['LOCATION', 'XCOORD', 'YCOORD']]
location_df.head(3)

,LOCATION,XCOORD,YCOORD
0,600 BLK KENILWORTH AVE NE S/B,404478.832866,136788.897372
1,5800 BLK NEW HAMPSHIRE AVE NE SW/B,399529.150000,143757.600000
2,3RD ST TUNNEL NW N/B BY MA AVE,398788.156100,136798.660300


In [9]:
# Convert xcoord and ycoord to Longitude/Latitude using the pyproj module
inProj = Proj(init='epsg:3559')
outProj = Proj(init='epsg:4326')

def convertCoords(row):
    return pd.Series(transform(inProj, outProj, row['XCOORD'], row['YCOORD']))

convert_df = mv_dropna.apply(convertCoords, axis=1)

convert_df.head(3)

,0,1
0,-76.948368,38.898940
1,-77.005433,38.961727
2,-77.013970,38.899039


In [10]:
lat_lon = convert_df.rename(columns={0:'LONGITUDE',1:'LATITUDE'})
lat_lon.head(3)

,LONGITUDE,LATITUDE
0,-76.948368,38.898940
1,-77.005433,38.961727
2,-77.013970,38.899039


In [11]:
# Group by location and calculate the total fine for each location
finepaid_df = mv_dropna.groupby('LOCATION')['TOTALPAID'].sum()
finepaid_df.head()

LOCATION
1 COLUMBUS CIR  NE        25
1 RHODE ISLAND AVE NE    150
1/MISSOURI AVE NW          0
100 12ST NW                0
100 15TH ST NW             0
Name: TOTALPAID, dtype: int64

In [16]:
# Top 10 location with the highest fine
highest_fine_df = finepaid_df.sort_values(ascending=False).head(10)
highest_fine_df

LOCATION
600 BLK KENILWORTH AVE NE S/B         314725
DC295 SW .7 MILES S/O EXIT 1 S/B      266430
DC295 SW .2 MILES S/O EXIT 1 N/B      264040
3RD ST TUNNEL NW N/B BY MA AVE        256265
2200 BLOCK K ST NW E/B                246785
2200 BLK K ST NW W/B                  224020
600 BLK NEW YORK AVENUE NE W/B        193250
DC 295 .4 MI S/O PENN AVE SE SW/B     169060
100 BLK MICHIGAN AVE NW E/B           165270
4600 BLK MASSACHUSETTS AVE NW NW/B    134825
Name: TOTALPAID, dtype: int64

In [17]:
# Merging the dataframes lat_lon and mv_dropna together with an outer join
moving_merge=mv_dropna.merge(lat_lon, how='outer',left_index=True, right_index=True)
moving_merge.head(3)

,STREETSEGID,LOCATION,XCOORD,YCOORD,TOTALPAID,VIOLATIONDESC,LONGITUDE,LATITUDE
0,11963.0,600 BLK KENILWORTH AVE NE S/B,404478.832866,136788.897372,200,SPEED 21-25 MPH OVER THE SPEED LIMIT,-76.948368,38.898940
1,838.0,5800 BLK NEW HAMPSHIRE AVE NE SW/B,399529.150000,143757.600000,100,SPEED 11-15 MPH OVER THE SPEED LIMIT,-77.005433,38.961727
2,6364.0,3RD ST TUNNEL NW N/B BY MA AVE,398788.156100,136798.660300,150,SPEED 16-20 MPH OVER THE SPEED LIMIT,-77.013970,38.899039


In [18]:
# Taking out the columns needed for futher work
moving_merge = moving_merge[['STREETSEGID','LOCATION','LATITUDE','LONGITUDE','VIOLATIONDESC','TOTALPAID']]
moving_merge.head(3)

,STREETSEGID,LOCATION,LATITUDE,LONGITUDE,VIOLATIONDESC,TOTALPAID
0,11963.0,600 BLK KENILWORTH AVE NE S/B,38.898940,-76.948368,SPEED 21-25 MPH OVER THE SPEED LIMIT,200
1,838.0,5800 BLK NEW HAMPSHIRE AVE NE SW/B,38.961727,-77.005433,SPEED 11-15 MPH OVER THE SPEED LIMIT,100
2,6364.0,3RD ST TUNNEL NW N/B BY MA AVE,38.899039,-77.013970,SPEED 16-20 MPH OVER THE SPEED LIMIT,150


In [19]:
# Renaming the moving_violations dataframe
M_violations= moving_merge.rename(columns={'STREETSEGID':'STREET_ID','VIOLATIONDESC':'VIOLATION_DESCRIPTION','TOTALPAID':'TOTAL_PAID'})
M_violations.head(3)

,STREET_ID,LOCATION,LATITUDE,LONGITUDE,VIOLATION_DESCRIPTION,TOTAL_PAID
0,11963.0,600 BLK KENILWORTH AVE NE S/B,38.898940,-76.948368,SPEED 21-25 MPH OVER THE SPEED LIMIT,200
1,838.0,5800 BLK NEW HAMPSHIRE AVE NE SW/B,38.961727,-77.005433,SPEED 11-15 MPH OVER THE SPEED LIMIT,100
2,6364.0,3RD ST TUNNEL NW N/B BY MA AVE,38.899039,-77.013970,SPEED 16-20 MPH OVER THE SPEED LIMIT,150


In [20]:
# Save M_violations in a CSV file
M_violations.to_csv('cleaned_moving_violations.csv')

In [21]:
# Transform the dataframe in format with columns used for constructing geojson object
# save the Geometry as feature
features = M_violations.apply(
    lambda row: Feature(geometry=Point((float(row['LONGITUDE']), float(row['LATITUDE'])))),
    axis=1).tolist()

# all the other columns used as properties
properties = M_violations.drop(['LONGITUDE', 'LATITUDE'], axis=1).to_dict('records')

# whole geojson object
feature_collection = FeatureCollection(features=features, properties=properties)

In [22]:
with open('mv.geojson', 'w', encoding='utf-8') as f:
    json.dump(feature_collection, f, ensure_ascii=False)

In [30]:
# Create Engine
engine = create_engine("sqlite:///moving_violations.sqlite", pool_pre_ping=True)

In [31]:
# Use `declarative_base` from SQLAlchemy to model the moving violations table as an ORM class
Base = declarative_base()

In [32]:
# Define the ORM class for `Moving violations`
class Movingviolations(Base):
    
    __tablename__ = 'Movingviolations'

    id = Column(Integer, primary_key=True)
    STREET_ID = Column(Float)
    LOCATION = Column(Text)
    LATITUDE = Column(Float)
    LONGITUDE = Column(Float)
    VIOLATION_DESCRIPTION = Column(Text)
    TOTAL_PAID = Column(Integer)
        
    def __repr__(self):
        return f"id={self.id}, name={self.station}"

In [33]:
# Use `create_all` to create the tables
Base.metadata.create_all(engine)

In [34]:
# Verify that the table names exist in the database
engine.table_names()

['Movingviolations']

In [35]:
# Use Pandas to Bulk insert each CSV file into their appropriate table
def populate_table(engine, table, csvfile):
    """Populates a table from a Pandas DataFrame."""
    # connect to the database
    conn = engine.connect()
    
    # Load the CSV file into a pandas dataframe 
    df_of_data_to_insert = pd.read_csv(csvfile)
    
    # Orient='records' creates a list of data to write
    # http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
    data = df_of_data_to_insert.to_dict(orient='records')

    # Optional: Delete all rows in the table 
    conn.execute(table.delete())

    # Insert the dataframe into the database in one bulk insert
    conn.execute(table.insert(), data)
    
# Call the function to insert the data for each table
populate_table(engine, Movingviolations.__table__, 'cleaned_moving_violations.csv')

OperationalError: (sqlite3.OperationalError) table Movingviolations has no column named STREET_ID [SQL: 'INSERT INTO "Movingviolations" ("STREET_ID", "LOCATION", "LATITUDE", "LONGITUDE", "VIOLATION_DESCRIPTION", "TOTAL_PAID") VALUES (?, ?, ?, ?, ?, ?)'] [parameters: ((11963.0, '600 BLK KENILWORTH AVE NE S/B', 38.89893998087359, -76.94836791552189, 'SPEED 21-25 MPH OVER THE SPEED LIMIT', 200), (838.0, '5800 BLK NEW HAMPSHIRE AVE NE SW/B', 38.96172748285357, -77.00543275070984, 'SPEED 11-15 MPH OVER THE SPEED LIMIT', 100), (6364.0, '3RD ST TUNNEL NW N/B BY MA AVE', 38.8990385034916, -77.01397018192333, 'SPEED 16-20 MPH OVER THE SPEED LIMIT', 150), (9292.0, '2000 BLK BRANCH AVE SE S/B', 38.86521441878462, -76.95909935475638, 'SPEED 11-15 MPH OVER THE SPEED LIMIT', 100), (14658.0, '600 BLK NEW YORK AVENUE NE W/B', 38.91232449669349, -76.99560642303777, 'SPEED 11-15 MPH OVER THE SPEED LIMIT', 100), (4361.0, '2200 BLOCK K ST NW E/B', 38.90252596079553, -77.04997741040427, 'SPEED 11-15 MPH OV

In [29]:
# Use a basic query to validate that the data was inserted correctly for table `Moving violations`
engine.execute("SELECT * FROM Movingviolations LIMIT 1").fetchall()

[(1, None, None, None, None, None, None)]